In [ ]:
from ibm_quantum_widgets import CircuitComposer
from IPython.display import display, Javascript, HTML
from qiskit.circuit import QuantumCircuit
from qiskit import transpile
from qiskit.providers.aer import QasmSimulator
from random import randint

In [ ]:
backend = QasmSimulator()

In [ ]:
JsWindowState.set("orchestrator-url", "http://localhost:8000")
JsWindowState.set("orchestrator-token", "qoffee")

In [ ]:
### VIEW: welcome
# initialize drink variable
drink = "coffee"
html = HTML("""
<h1>Welcome</h1>
<br/>
<br/>
<button class="btn btn-primary" data-q-target="circuitSelection">Get your coffee</button>
<button class="btn btn-primary" >Get more information</button>
""")
display(html)

In [ ]:
# build some sample circuits
qc0 = QuantumCircuit(2, 2)

qc1 = QuantumCircuit(2, 2)
qc1.h(0)
qc1.h(1)

In [ ]:
qcreg = {
    'scratch': qc0,
    'superposition': qc1,
}

In [ ]:
# initialize qc variable
qc = qcreg['scratch'].copy()

In [ ]:
### VIEW: circuitSelection
html = HTML("""
<h1>Where do you want to start</h1>
<br/>
<br/>
<button class="btn btn-primary" data-q-code="qc=qcreg['scratch'].copy()" data-q-target="initEditor,editor">Start from scratch</button>
<button class="btn btn-primary" data-q-code="qc=qcreg['superposition'].copy()" data-q-target="initEditor,editor">Start with superposition</button>
<button class="btn btn-primary">...</button>
<br><br>
<button class="btn btn-default" data-q-target="welcome">Back</button>
""")
display(html)

In [ ]:
backend.run?

In [ ]:
from random import randint

In [ ]:
randint(0, 2000)

In [ ]:
def get_counts(num_shots):
    # copy current circuit
    mqc = editor.circuit.copy()
    # add measurement
    mqc.measure(range(2), range(2))
    # get simulation result
    mqc_compiled = transpile(mqc, backend)
    job_sim = backend.run(mqc_compiled, shots=num_shots, seed=randint(0, 2500))
    result_sim = job_sim.result()
    counts = result_sim.get_counts(mqc_compiled)
    return counts

In [ ]:
### VIEW: initEditor
editor = CircuitComposer(circuit=qc)

# handler for change events
def calculate(change_event):
    # there are multiple change events, only use one
    if change_event['name'] != "circuit":
        return
    counts = get_counts(1024)
    for bit in counts:
        JsWindowState.set("editor-circuit-bit-"+bit, counts[bit]/1024)
    qasm = editor.circuit.qasm()
    JsWindowState.set("editor-circuit-qasm", qasm)
    
# add handler to change events
editor.observe(calculate)

In [ ]:
### VIEW: editor
editor

In [ ]:
### VIEW: editor
html = HTML("""
Coffee (00):
<div class="progress editor-circuit-probbar-00">
  <div class="progress-bar editor-circuit-probtext-00" role="progressbar" style="width: 0%" aria-valuenow="0" aria-valuemin="0" aria-valuemax="100">0%</div>
</div>
Cappucino (01):
<div class="progress editor-circuit-probbar-01">
  <div class="progress-bar editor-circuit-probtext-01" role="progressbar" style="width: 0%" aria-valuenow="0" aria-valuemin="0" aria-valuemax="100">0%</div>
</div>
Espresso (10):
<div class="progress editor-circuit-probbar-10">
  <div class="progress-bar editor-circuit-probtext-10" role="progressbar" style="width: 0%" aria-valuenow="0" aria-valuemin="0" aria-valuemax="100">0%</div>
</div>
Latte Macchiato (11):
<div class="progress editor-circuit-probbar-11">
  <div class="progress-bar editor-circuit-probtext-11" role="progressbar" style="width: 0%" aria-valuenow="0" aria-valuemin="0" aria-valuemax="100">0%</div>
</div>


<button class="btn btn-primary" data-q-target="order">Get your drink</button>
<button class="btn btn-primary" data-q-open-external="editor-circuit-qasm">Transfer to IBM Quantum Composer</button>
<button class="btn btn-default" data-q-target="circuitSelection">Back</button>
""")
display(html) 

In [ ]:
def set_new_drink():
    global a
    a = randint(0, 1000)
    counts = get_counts(1)
    JsWindowState.set("editor-circuit-single-shot-bit", list(counts.keys())[0])

In [ ]:
### VIEW: singleShot
set_new_drink()

In [ ]:
### VIEW: order
html = HTML("""
<h1>Chances are good to get: <span data-q-html>${editor-circuit-max-name}</span></h1>
<br/>
<p>Actual measurement: <span data-q-html>${editor-circuit-single-shot-name}</span></p>
<br/>
<button class="btn btn-primary" data-q-execute="singleShot">Run Again</button>
<button class="btn btn-primary" data-q-fn="requestDrink">Okay, prepare</button>
<button class="btn btn-default" data-q-target="editor">Back</button>
""")
display(html)

In [ ]:
### VIEW: drinkPreparation
html = HTML("""
<h1 data-q-html>Please wait while we are preparing your ${editor-circuit-single-shot-name} ☕</h1>
""")
display(html)

In [ ]:
### VIEW: success
html = HTML("""
<h1 data-q-html>Thank you and enjoy your ${editor-circuit-single-shot-name} ☕</h1>
<button class="btn btn-default" data-q-target="welcome">Back</button>
""")
display(html)

In [ ]:
### VIEW: error
html = HTML("""
<h1>Something went wrong :/</h1>
<p>Please contact someone around you to help</p>
<button class="btn btn-default" data-q-target="welcome">Back</button>
""")
display(html) 